In [ ]:

!pip install ultralytics --quiet
!pip install seaborn --quiet  #juste pour les plots


In [ ]:

import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests
import pandas as pd
from PIL import Image
import os, shutil
from ultralytics import YOLO

#je sais pas c quoi des pickle
#normalement les labels sont deja au format YOLO


data_train_features_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module7/exercise/X_train.pkl'
data_train_labels_url   = 'https://www.raphaelcousin.com/modules/data-science-practice/module7/exercise/y_train.pkl'
data_test_features_url  = 'https://www.raphaelcousin.com/modules/data-science-practice/module7/exercise/X_test.pkl'

def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name}')


download_file(data_train_features_url, 'X_train.pkl')
download_file(data_train_labels_url, 'y_train.pkl')
download_file(data_test_features_url, 'X_test.pkl')


with open('X_train.pkl', 'rb') as f:
    X = pickle.load(f)

with open('y_train.pkl', 'rb') as f:
    y = pickle.load(f)

with open('X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

print(f"Loaded {len(X)} training images")
print(f"Sample image shape: {X[0].shape}")
print(f"Sample labels shape: {y[0].shape}")
print(f"Sample labels (YOLO format):\n{y[0]}")
print(f"Loaded {len(X_test)} test images")


In [ ]:
#c pour visualiser
#est-ce que y correspond bien aux bateaux
def plot_image_with_boxes(image, boxes, ax=None):
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(image)
    height, width = image.shape[:2]
    for box in boxes:
        class_id, x_center, y_center, box_width, box_height = box
        x_center_px = x_center * width
        y_center_px = y_center * height
        box_width_px = box_width * width
        box_height_px = box_height * height
        x1 = x_center_px - box_width_px / 2
        y1 = y_center_px - box_height_px / 2
        rect = patches.Rectangle((x1, y1), box_width_px, box_height_px,
                                 linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)
        ax.text(x1, y1-5, 'boat', color='pink', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))
    ax.axis('off')
    return ax

#test avec 6 images

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()
for i in range(6):
    plot_image_with_boxes(X[i], y[i], ax=axes[i])
    axes[i].set_title(f'Image {i}: {len(y[i])} boat(s)')
plt.tight_layout()
plt.show()


In [ ]:
#et donc apparemment yolo n'accepte pas les pkl donc faut transformer avec un dossier image+labels+un fichier data qui dit ou sont les images
import os, shutil


if os.path.exists("boat_dataset"):
    shutil.rmtree("boat_dataset")
os.makedirs("boat_dataset/images/train", exist_ok=True)
os.makedirs("boat_dataset/labels/train", exist_ok=True)
os.makedirs("boat_dataset/images/val", exist_ok=True)
os.makedirs("boat_dataset/labels/val", exist_ok=True)


split_idx = int(0.8 * len(X))
X_train_split, y_train_split = X[:split_idx], y[:split_idx]
X_val_split, y_val_split = X[split_idx:], y[split_idx:]

def save_split(X_split, y_split, split_name):
    for i, (img, boxes) in enumerate(zip(X_split, y_split)):
        img_path = f"boat_dataset/images/{split_name}/{i}.jpg"
        label_path = f"boat_dataset/labels/{split_name}/{i}.txt"


        pil_img = Image.fromarray(img)
        if pil_img.mode == "RGBA":
            pil_img = pil_img.convert("RGB")

        pil_img.save(img_path)


        with open(label_path, "w") as f:
            for box in boxes:
                class_id, x_center, y_center, w, h = box
                f.write(f"{int(class_id)} {x_center} {y_center} {w} {h}\n")


save_split(X_train_split, y_train_split, "train")
save_split(X_val_split, y_val_split, "val")


dataset_path = os.path.abspath("boat_dataset")

with open("boat_dataset/data.yaml", "w") as f:
    f.write(
        f"train: {dataset_path}/images/train\n"
        f"val: {dataset_path}/images/val\n"
        "nc: 1\n"
        "names: ['boat']\n"
    )


print(f"Train images: {len(X_train_split)}, Val images: {len(X_val_split)}")
print("Contenu data.yaml :")
print(open("boat_dataset/data.yaml").read())


In [ ]:
#je charge un modèle pré entraîné
model = YOLO('yolov8s.pt')



results = model.train(
    data="boat_dataset/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    lr0=5e-4,
    patience=10,
    device=0
)




In [ ]:
#pour voir si j'ai bien dépasser 0.73
val_results = model.val(data="boat_dataset/data.yaml", imgsz=640, split="val")

print("resultats validation locale:")
print(f"mAP50:    {val_results.box.map50:.3f}")



In [ ]:


predictions = []

for img_idx, img in enumerate(X_test):
    temp_img = Image.fromarray(img)
    if temp_img.mode == "RGBA":  #pour pas que ça plante
        temp_img = temp_img.convert("RGB")
    temp_path = f"temp_{img_idx}.jpg"
    temp_img.save(temp_path)

    results = model.predict(source=temp_path, conf=0.25, verbose=False)

    for box_idx, box in enumerate(results[0].boxes):
        x_center, y_center, width, height = box.xywhn[0].cpu().numpy()
        confidence = box.conf[0].cpu().numpy()
        class_id = int(box.cls[0].cpu().numpy())
        predictions.append({
            'image_id': img_idx,
            'box_idx': box_idx,
            'class_id': 0,
            'confidence': float(confidence),
            'x_center': float(x_center),
            'y_center': float(y_center),
            'width': float(width),
            'height': float(height)
        })

    if (img_idx+1) % 10 == 0:
        print(f"Processed {img_idx+1}/{len(X_test)} images")

df_predictions = pd.DataFrame(predictions)
df_predictions.to_csv('predictions.csv', index=False)


